In [62]:
inputs2 = tokenizer(sentences, padding=True, truncation=True, return_tensors="tf", max_length=20)
inputs2['input_ids'][0]

<tf.Tensor: shape=(12,), dtype=int32, numpy=
array([  101,  1996,  4248,  2829,  4419, 14523,  2058,  1996, 13971,
        3899,  1012,   102], dtype=int32)>

In [65]:
input_ids2 = inputs2.input_ids.numpy()
input_ids2[0]

array([  101,  1996,  4248,  2829,  4419, 14523,  2058,  1996, 13971,
        3899,  1012,   102], dtype=int32)

In [72]:
mask2 = np.random.rand(*input_ids2.shape) < 0.2
mask2[0]

array([False,  True, False, False, False, False,  True, False, False,
       False, False, False])

In [75]:
labels2 = -100 * np.ones(input_ids2.shape, dtype=int)
labels2[0]

array([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100])

In [77]:
labels2[mask2] = input_ids2[mask2]
labels2[0]

array([-100, 1996, -100, -100, -100, -100, 2058, -100, -100, -100, -100,
       -100])

In [79]:
input_ids2[mask2] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)
input_ids2[0]

array([  101,   103,  4248,  2829,  4419, 14523,   103,  1996, 13971,
        3899,  1012,   102], dtype=int32)

In [1]:
from transformers import BertTokenizer, TFBertForMaskedLM
import tensorflow as tf
import numpy as np

# Load pre-trained model and tokenizer (without weights)
config = TFBertForMaskedLM.config_class.from_pretrained('bert-base-uncased')
model = TFBertForMaskedLM(config)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Sample data
sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "I love to eat pizza.",
    "The weather is nice today."
]

# Tokenize input data and create a mask array
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="tf", max_length=20)
input_ids = inputs.input_ids.numpy()
mask = np.random.rand(*input_ids.shape) < 0.15  # 15% probability of masking a token
labels = -100 * np.ones(input_ids.shape, dtype=int)  # Initialize labels to -100
labels[mask] = input_ids[mask]  # Assign labels for masked tokens
print(f"Labels: {labels}")
# Apply mask
input_ids[mask] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)
print(f"Raw tokens going in where 103 is a mask: {input_ids}")
# Prepare dataset
dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": input_ids, "attention_mask": inputs.attention_mask, "labels": labels}
)).batch(2)


def masked_sparse_categorical_crossentropy(y_true, y_pred):
    # Create a mask to ignore the -100 labels
    mask = tf.cast(tf.not_equal(y_true, -100), tf.float32)
    
    # Update y_true to a valid label value (e.g., 0) where it's -100
    y_true = tf.where(tf.not_equal(y_true, -100), y_true, tf.zeros_like(y_true))

    # Calculate the sparse categorical crossentropy
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

    # Apply the mask
    loss *= mask

    # Return the mean loss, but avoid division by zero
    return tf.reduce_sum(loss) / tf.maximum(tf.reduce_sum(mask), 1)

# Compile the model with the custom loss function
model.compile(optimizer='adam', loss=masked_sparse_categorical_crossentropy)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=masked_sparse_categorical_crossentropy)

# Train the model
model.fit(dataset, epochs=10)


2023-11-29 13:52:19.068628: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-29 13:52:19.068713: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-29 13:52:19.104554: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 13:52:20.021366: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-29 13:52:20.021462: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

2023-11-29 13:52:24.579979: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-11-29 13:52:24.580844: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-29 13:52:24.580909: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (george-ThinkPad-X220-Tablet): /proc/driver/nvidia/version does not exist


Labels: [[ -100  -100  -100  2829  -100  -100  -100  -100 13971  -100  -100  -100]
 [ -100  -100  -100  2000  -100  -100  1012  -100     0  -100  -100  -100]
 [ -100  -100  4633  2003  -100  2651  -100  -100  -100  -100  -100  -100]]
Raw tokens going in where 103 is a mask: [[  101  1996  4248   103  4419 14523  2058  1996   103  3899  1012   102]
 [  101  1045  2293   103  4521 10733   103   102   103     0     0     0]
 [  101  1996   103   103  3835   103  1012   102     0     0     0     0]]
Epoch 1/10
2/2 [==============================] - 28s 697ms/step - loss: 10.4796
Epoch 2/10
2/2 [==============================] - 2s 693ms/step - loss: 7.6194
Epoch 3/10
2/2 [==============================] - 2s 702ms/step - loss: 6.6910
Epoch 4/10
2/2 [==============================] - 2s 682ms/step - loss: 5.8492
Epoch 5/10
2/2 [==============================] - 2s 677ms/step - loss: 5.1625
Epoch 6/10
2/2 [==============================] - 2s 687ms/step - loss: 4.4735
Epoch 7/10
2/2 [=======

In [2]:
output = model.predict(dataset)

2/2 [==============================] - 4s 153ms/step


In [3]:
output.logits[0].shape

(12, 30522)

In [4]:
model.summary()

Model: "tf_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108891648 
                                                                 
 mlm___cls (TFBertMLMHead)   multiple                  24459834  
                                                                 
Total params: 109,514,298
Trainable params: 109,514,298
Non-trainable params: 0
_________________________________________________________________
